In [1]:
# Import packages

import pandas as pd 
import numpy as np 
import os

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

import statsmodels.api as sm
from scipy import stats

from joblib import dump, load

pd.set_option('display.max_columns', None)

In [2]:
nfl = pd.read_csv('../../data/value_models_combined.csv')
nfl_rolling = pd.read_csv('../../data/value_models_combined_6_game_rolling.csv')

In [3]:
# Want to adjust the qb rating to take into account how good the opposing defense they faced was
# Example: Two qbs have an epa of 10 in a game. But one qb played a team that normally gives up 20 epa
# The other qb played a team that normally gives up 0 epa. By the current model, both qbs had equivalent games, 
# but in reality the qb that played a tougher defense had the better performance when defense is taken into account

In [4]:
qb = nfl[['season', 'week', 'team', 'opponent', 'team_full', 'opponent_full', 'score', 'opponent_score', 
          'qb', 'qb_value']]
opposing_def = nfl_rolling[['season', 'week', 'team_full', 
          'qb_def_value']]

In [5]:
df = qb.merge(opposing_def, left_on=['season', 'week', 'opponent_full'], 
             right_on=['season', 'week', 'team_full'], suffixes = ('','_opponent'))
df = df.drop(columns=['team_full_opponent'])

In [6]:
df

,season,week,team,opponent,team_full,opponent_full,score,opponent_score,qb,qb_value,qb_def_value
0,2014,1,ARI,LAC,Arizona Cardinals,Los Angeles Chargers,18,17,C. Palmer,0.491486,NaN
1,2014,1,ATL,NO,Atlanta Falcons,New Orleans Saints,37,34,M. Ryan,1.997776,NaN
2,2014,1,BAL,CIN,Baltimore Ravens,Cincinnati Bengals,16,23,J. Flacco,-0.509634,NaN
3,2014,1,BUF,CHI,Buffalo Bills,Chicago Bears,23,20,E. Manuel,-0.171272,NaN
4,2014,1,CAR,TB,Carolina Panthers,Tampa Bay Buccaneers,20,14,D. Anderson,0.728679,NaN
...,...,...,...,...,...,...,...,...,...,...,...
4857,2022,21,KC,CIN,Kansas City Chiefs,Cincinnati Bengals,23,20,P. Mahomes,0.500521,0.509250
4858,2022,21,PHI,SF,Philadelphia Eagles,San Francisco 49ers,31,7,J. Hurts,-0.148775,0.407030
4859,2022,21,SF,PHI,San Francisco 49ers,Philadelphia Eagles,7,31,J. Johnson,-0.835878,0.398366
4860,2022,22,KC,PHI,Kansas City Chiefs,Philadelphia Eagles,38,35,P. Mahomes,1.243585,0.601635


In [7]:
# Not sure the best way to adjust ratings
# One possibility - qb_value over expected, which would be qb_value - qb_def_value. However, this is a huge adjustment
# Another possibility is a small adjustment to the qb_value based on the qb value over expected, which I think
# makes the most sense. 

# Implementing the above:
# There are going to be two adjustable thresholds
# original_value_percentage: the amount of the original value that should be kept
# adjustment_threshold: how much more value do we want to add or take away for the performance based on 
# the value over or under expected
# original value percentage of 1 and adjustment threshold of 0 means value is unchanged
# original value percentage of 0 and adjustment threshold of 1 means the value is entirely based on the value over expected
# Will likely need to mess around with these values and see what works the best 


def value_adjustment(value, adjusting_value, original_value_percentage=.95, adjustment_threshold=.1):
    value_over_expected = value + adjusting_value
    adjusted_value = (value_over_expected*adjustment_threshold) + (value*original_value_percentage)
    adjustment = adjusted_value - value
    
    return adjustment, adjusted_value

df[['qb_adjustment', 'qb_adjusted_value']] = df.apply(lambda x: value_adjustment(x.qb_value, x.qb_def_value, 
                                original_value_percentage=.7, adjustment_threshold=.3), axis=1, result_type='expand')

In [8]:
# Save df here later
df.to_csv('qb_value_modeling_adjusted_data/qb_values_with_adjustment.csv')

In [9]:
testing = df.dropna()
testing.sort_values(by='qb_adjustment', ascending=False)

,season,week,team,opponent,team_full,opponent_full,score,opponent_score,qb,qb_value,qb_def_value,qb_adjustment,qb_adjusted_value
1660,2017,2,TEN,JAX,Tennessee Titans,Jacksonville Jaguars,37,16,M. Mariota,0.198672,3.467329,1.040199,1.238870
1637,2017,2,CLE,BAL,Cleveland Browns,Baltimore Ravens,10,24,D. Kizer,-1.631672,2.724325,0.817298,-0.814374
2166,2018,2,CIN,BAL,Cincinnati Bengals,Baltimore Ravens,34,23,A. Dalton,0.641299,2.582997,0.774899,1.416198
2878,2019,8,CLE,NE,Cleveland Browns,New England Patriots,13,27,B. Mayfield,-0.928611,2.368926,0.710678,-0.217933
2748,2019,3,NYJ,NE,New York Jets,New England Patriots,14,30,L. Falk,-1.774771,2.356517,0.706955,-1.067816
...,...,...,...,...,...,...,...,...,...,...,...,...,...
2732,2019,3,DAL,MIA,Dallas Cowboys,Miami Dolphins,31,6,D. Prescott,0.364711,-2.246169,-0.673851,-0.309139
2167,2018,2,CLE,NO,Cleveland Browns,New Orleans Saints,18,21,T. Taylor,-0.075865,-2.540035,-0.762011,-0.837875
2695,2019,2,BUF,NYG,Buffalo Bills,New York Giants,28,14,J. Allen,0.569414,-2.654348,-0.796304,-0.226890
4346,2022,2,LV,ARI,Las Vegas Raiders,Arizona Cardinals,23,29,D. Carr,0.184271,-2.705768,-0.811730,-0.627459


In [10]:
testing[(testing.season==2022) & (testing.team_full=='San Francisco 49ers')]

,season,week,team,opponent,team_full,opponent_full,score,opponent_score,qb,qb_value,qb_def_value,qb_adjustment,qb_adjusted_value
4356,2022,2,SF,SEA,San Francisco 49ers,Seattle Seahawks,27,7,J. Garoppolo,0.339361,-1.121366,-0.336410,0.002951
4388,2022,3,SF,DEN,San Francisco 49ers,Denver Broncos,10,11,J. Garoppolo,-1.224562,0.393242,0.117972,-1.106590
4420,2022,4,SF,LA,San Francisco 49ers,Los Angeles Rams,24,9,J. Garoppolo,0.835769,-0.164655,-0.049397,0.786372
4452,2022,5,SF,CAR,San Francisco 49ers,Carolina Panthers,37,15,J. Garoppolo,1.464973,0.275055,0.082517,1.547490
4481,2022,6,SF,ATL,San Francisco 49ers,Atlanta Falcons,14,28,J. Garoppolo,0.140460,-0.657147,-0.197144,-0.056684
4508,2022,7,SF,KC,San Francisco 49ers,Kansas City Chiefs,23,44,J. Garoppolo,0.035859,-0.194913,-0.058474,-0.022615
4537,2022,8,SF,LA,San Francisco 49ers,Los Angeles Rams,31,14,J. Garoppolo,1.182658,0.153740,0.046122,1.228780
4590,2022,10,SF,LAC,San Francisco 49ers,Los Angeles Chargers,22,16,J. Garoppolo,0.792546,0.016583,0.004975,0.797521
4619,2022,11,SF,ARI,San Francisco 49ers,Arizona Cardinals,38,10,J. Garoppolo,1.320104,0.255564,0.076669,1.396773
4650,2022,12,SF,NO,San Francisco 49ers,New Orleans Saints,13,0,J. Garoppolo,-0.156994,-0.070199,-0.021060,-0.178054


In [11]:
testing.sort_values(by='qb_adjusted_value', ascending=False)

,season,week,team,opponent,team_full,opponent_full,score,opponent_score,qb,qb_value,qb_def_value,qb_adjustment,qb_adjusted_value
237,2014,8,PIT,IND,Pittsburgh Steelers,Indianapolis Colts,51,34,B. Roethlisberger,3.475095,0.673309,0.201993,3.677088
3271,2020,3,KC,BAL,Kansas City Chiefs,Baltimore Ravens,34,20,P. Mahomes,3.024486,0.724304,0.217291,3.241777
2797,2019,5,HOU,ATL,Houston Texans,Atlanta Falcons,53,32,D. Watson,3.100802,-0.543057,-0.162917,2.937885
4180,2021,16,CIN,BAL,Cincinnati Bengals,Baltimore Ravens,41,21,J. Burrow,2.971832,-0.150829,-0.045249,2.926583
2175,2018,2,KC,PIT,Kansas City Chiefs,Pittsburgh Steelers,42,37,P. Mahomes,2.514604,1.235829,0.370749,2.885353
...,...,...,...,...,...,...,...,...,...,...,...,...,...
3560,2020,13,LAC,NE,Los Angeles Chargers,New England Patriots,0,45,J. Herbert,-2.814933,-0.699435,-0.209831,-3.024764
1823,2017,8,MIA,BAL,Miami Dolphins,Baltimore Ravens,0,40,M. Moore,-3.126910,0.102239,0.030672,-3.096238
4229,2021,17,NYG,CHI,New York Giants,Chicago Bears,3,29,M. Glennon,-3.402921,-0.053857,-0.016157,-3.419078
2808,2019,5,NYJ,PHI,New York Jets,Philadelphia Eagles,6,31,L. Falk,-3.372992,-0.826012,-0.247804,-3.620796


In [12]:
testing.groupby(by=['season','qb']).mean().sort_values(by='qb_value', ascending=False)[0:30]

week      score  opponent_score  qb_value  \
season qb                                                                  
2020   M. Mariota         15.000000  27.000000       30.000000  1.583775   
2016   J. Garoppolo        2.000000  31.000000       24.000000  1.428417   
2019   M. Schaub           8.000000  20.000000       27.000000  1.421588   
2020   J. Garoppolo        5.000000  29.333333       11.666667  1.135910   
2018   M. Barkley         10.000000  41.000000       10.000000  1.063228   
       P. Mahomes         10.529412  34.647059       26.058824  1.062260   
2020   P. Mahomes         10.882353  28.647059       22.117647  0.970709   
2018   K. Allen           17.000000  33.000000       14.000000  0.932754   
2016   M. Ryan            11.166667  34.666667       25.000000  0.925676   
2021   J. Johnson         12.500000  25.500000       43.000000  0.907915   
2019   P. Mahomes         11.733333  29.866667       19.800000  0.907834   
2022   P. Mahomes         11.842105  28.421053       22.263158  0.895449   
2016   T. Brady           12.933333  30.933333       16.666667  0.876759   
2020   A. Rodgers         10.941176  30.823529       22.588235  0.873644   
2021   J. Allen           11.117647  31.764706       18.294118  0.872650   
2014   A. Rodgers         10.705882  30.470588       21.235294  0.865107   
2020   C. Henne           17.000000  21.000000       38.000000  0.858214   
2017   J. Garoppolo       15.000000  28.800000       19.800000  0.799191   
2018   B. Roethlisberger   9.666667  27.133333       22.600000  0.790162   
2019   L. Jackson          9.733333  30.400000       19.333333  0.784932   
2020   D. Prescott         3.500000  36.500000       40.000000  0.778737   
2015   L. McCown           3.000000  22.000000       27.000000  0.768115   
2016   A. Rodgers         11.388889  27.666667       25.166667  0.727577   
2014   T. Romo            10.562500  29.875000       21.375000  0.726642   
2016   D. Prescott         9.800000  28.000000       19.533333  0.717066   
2020   J. Allen           11.000000  30.111111       23.500000  0.711290   
2018   D. Brees           10.500000  30.812500       19.500000  0.705202   
2014   T. Brady           10.882353  32.176471       19.117647  0.704528   
2018   M. Ryan             9.600000  26.800000       27.000000  0.687988   
2020   M. Rudolph         17.000000  22.000000       24.000000  0.682109   

                          qb_def_value  qb_adjustment  qb_adjusted_value  
season qb                                                                 
2020   M. Mariota            -0.019556      -0.005867           1.577908  
2016   J. Garoppolo           0.841281       0.252384           1.680802  
2019   M. Schaub             -0.139806      -0.041942           1.379646  
2020   J. Garoppolo           0.061349       0.018405           1.154314  
2018   M. Barkley            -0.046157      -0.013847           1.049380  
       P. Mahomes             0.098557       0.029567           1.091827  
2020   P. Mahomes             0.092495       0.027748           0.998458  
2018   K. Allen               0.080587       0.024176           0.956930  
2016   M. Ryan                0.051661       0.015498           0.941174  
2021   J. Johnson             0.037502       0.011251           0.919165  
2019   P. Mahomes            -0.029604      -0.008881           0.898953  
2022   P. Mahomes             0.245187       0.073556           0.969005  
2016   T. Brady               0.121076       0.036323           0.913082  
2020   A. Rodgers            -0.221961      -0.066588           0.807056  
2021   J. Allen              -0.242225      -0.072667           0.799982  
2014   A. Rodgers             0.224096       0.067229           0.932336  
2020   C. Henne              -0.160465      -0.048140           0.810075  
2017   J. Garoppolo           0.297144       0.089143           0.888334  
2018   B. Roethlisberger     -0.080118      -0.024036           0.766127  
2019   L. Jackson      

In [13]:
testing.groupby(by=['season', 'qb']).mean().sort_values(by='qb_adjusted_value', ascending=False)[0:30]

week      score  opponent_score  qb_value  \
season qb                                                                  
2016   J. Garoppolo        2.000000  31.000000       24.000000  1.428417   
2020   M. Mariota         15.000000  27.000000       30.000000  1.583775   
2019   M. Schaub           8.000000  20.000000       27.000000  1.421588   
2020   J. Garoppolo        5.000000  29.333333       11.666667  1.135910   
2015   L. McCown           3.000000  22.000000       27.000000  0.768115   
2018   P. Mahomes         10.529412  34.647059       26.058824  1.062260   
       M. Barkley         10.000000  41.000000       10.000000  1.063228   
2020   P. Mahomes         10.882353  28.647059       22.117647  0.970709   
2022   P. Mahomes         11.842105  28.421053       22.263158  0.895449   
2018   K. Allen           17.000000  33.000000       14.000000  0.932754   
2016   M. Ryan            11.166667  34.666667       25.000000  0.925676   
2014   A. Rodgers         10.705882  30.470588       21.235294  0.865107   
2021   J. Johnson         12.500000  25.500000       43.000000  0.907915   
2016   T. Brady           12.933333  30.933333       16.666667  0.876759   
2019   P. Mahomes         11.733333  29.866667       19.800000  0.907834   
2017   J. Garoppolo       15.000000  28.800000       19.800000  0.799191   
2019   L. Jackson          9.733333  30.400000       19.333333  0.784932   
2020   C. Henne           17.000000  21.000000       38.000000  0.858214   
       A. Rodgers         10.941176  30.823529       22.588235  0.873644   
2021   J. Allen           11.117647  31.764706       18.294118  0.872650   
2014   T. Brady           10.882353  32.176471       19.117647  0.704528   
2018   B. Roethlisberger   9.666667  27.133333       22.600000  0.790162   
       D. Brees           10.500000  30.812500       19.500000  0.705202   
2016   D. Prescott         9.800000  28.000000       19.533333  0.717066   
       A. Rodgers         11.388889  27.666667       25.166667  0.727577   
2014   T. Romo            10.562500  29.875000       21.375000  0.726642   
2021   P. Mahomes         11.473684  29.210526       22.052632  0.665028   
2017   C. Wentz            7.833333  31.166667       19.416667  0.657715   
2020   J. Allen           11.000000  30.111111       23.500000  0.711290   
2021   A. Rodgers         10.500000  28.125000       20.812500  0.673188   

                          qb_def_value  qb_adjustment  qb_adjusted_value  
season qb                                                                 
2016   J. Garoppolo           0.841281       0.252384           1.680802  
2020   M. Mariota            -0.019556      -0.005867           1.577908  
2019   M. Schaub             -0.139806      -0.041942           1.379646  
2020   J. Garoppolo           0.061349       0.018405           1.154314  
2015   L. McCown              1.249160       0.374748           1.142863  
2018   P. Mahomes             0.098557       0.029567           1.091827  
       M. Barkley            -0.046157      -0.013847           1.049380  
2020   P. Mahomes             0.092495       0.027748           0.998458  
2022   P. Mahomes             0.245187       0.073556           0.969005  
2018   K. Allen               0.080587       0.024176           0.956930  
2016   M. Ryan                0.051661       0.015498           0.941174  
2014   A. Rodgers             0.224096       0.067229           0.932336  
2021   J. Johnson             0.037502       0.011251           0.919165  
2016   T. Brady               0.121076       0.036323           0.913082  
2019   P. Mahomes            -0.029604      -0.008881           0.898953  
2017   J. Garoppolo           0.297144       0.089143           0.888334  
2019   L. Jackson             0.110924       0.033277           0.818209  
2020   C. Henne              -0.160465      -0.048140           0.810075  
       A. Rodgers            -0.221961      -0.066588           0.807056  
2021   J. Allen        

In [14]:
testing.groupby(by=['season', 'qb']).mean().sort_values(by='qb_adjustment', ascending=False)[0:30]

week      score  opponent_score  qb_value  \
season qb                                                               
2019   C. McCoy         5.000000   7.000000       33.000000 -2.145695   
2015   L. McCown        3.000000  22.000000       27.000000  0.768115   
2022   J. Winston       2.500000  12.000000       21.000000 -0.709425   
2020   R. Griffin      12.000000  14.000000       19.000000 -1.734426   
2021   I. Book         16.000000   3.000000       20.000000 -2.872912   
2018   S. Bradford      2.500000   7.000000       25.000000 -1.018308   
2022   T. Siemian      12.000000  10.000000       31.000000 -0.955612   
2020   R. Finley       15.000000  27.000000       17.000000 -0.251764   
       C. Streveler    17.000000   7.000000       18.000000 -1.423029   
2016   J. Garoppolo     2.000000  31.000000       24.000000  1.428417   
2021   B. Allen        18.000000  16.000000       21.000000 -1.225659   
2014   J. Garoppolo    17.000000   9.000000       17.000000 -1.267725   
2017   E. Manuel        5.000000  17.000000       30.000000 -0.143480   
2019   R. Griffin III  17.000000  28.000000       10.000000 -0.732275   
2015   D. Orlovsky      5.000000  17.000000       42.000000 -0.244223   
2016   J. Cutler        7.750000  15.000000       24.250000 -0.709811   
2022   J. Stidham      17.500000  23.500000       34.000000 -0.152685   
2016   J. Brissett      3.500000  13.500000        8.000000 -0.760192   
2021   P. Walker       10.000000  34.000000       10.000000 -0.226768   
2014   R. Lindley      17.000000  13.000000       27.333333 -1.082438   
2016   C. Cook         17.500000  10.000000       25.500000 -1.871898   
2019   B. Mayfield      9.666667  21.466667       23.333333 -0.122478   
2020   G. Gilbert       9.000000  19.000000       24.000000 -0.556204   
2015   J. Manziel      11.000000  16.166667       22.000000 -0.207630   
2017   B. Petty        16.000000  10.666667       23.666667 -0.929331   
2021   C. Rush          8.000000  20.000000       16.000000 -0.060842   
2020   J. Hurts        15.500000  20.250000       27.750000 -0.235608   
2015   A. Luck          5.833333  23.833333       27.833333 -0.253530   
2017   B. Gabbert      13.000000  18.200000       22.800000 -1.206167   
       D. Kizer        10.153846  14.230769       26.076923 -0.810493   

                       qb_def_value  qb_adjustment  qb_adjusted_value  
season qb                                                              
2019   C. McCoy            2.061393       0.618418          -1.527277  
2015   L. McCown           1.249160       0.374748           1.142863  
2022   J. Winston          1.238571       0.371571          -0.337854  
2020   R. Griffin          1.156411       0.346923          -1.387503  
2021   I. Book             1.146971       0.344091          -2.528821  
2018   S. Bradford         1.018638       0.305591          -0.712717  
2022   T. Siemian          1.002780       0.300834          -0.654778  
2020   R. Finley           0.962683       0.288805           0.037041  
       C. Streveler        0.907172       0.272152          -1.150877  
2016   J. Garoppolo        0.841281       0.252384           1.680802  
2021   B. Allen            0.826635       0.247990          -0.977669  
2014   J. Garoppolo        0.792428       0.237728          -1.029997  
2017   E. Manuel           0.703971       0.211191           0.067712  
2019   R. Griffin III      0.673627       0.202088          -0.530187  
2015   D. Orlovsky         0.655085       0.196526          -0.047698  
2016   J. Cutler           0.644326       0.193298          -0.516513  
2022   J. Stidham          0.638452       0.191536           0.038851  
2016   J. Brissett         0.611360       0.183408          -0.576784  
2021   P. Walker           0.583827       0.175148          -0.051620  
2014   R. Lindley          0.565277       0.169583          -0.912855  
2016   C. Cook             0.564024       0.169207          -1.702691  
2019   B. Mayfield         0.524786